In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from io import StringIO
import requests
import re

# An Analysis of Movie Performance

In this part, you’ll gather data about popular movies and award winners. The goal is to build a dataset that you’ll later use to analyze what makes a movie successful and how awards and box office performance relate to one another.

### Part 1: Data Gathering
1. Scrape Best Picture Data.  
    * Scrape the [Best Picture wikipedia page](https://en.wikipedia.org/wiki/Academy_Award_for_Best_Picture).  
    * Extract for each year:  
        * Year  
        * Film Title  
        * Winner (Yes/No)  
    * Data cleaning tips:  
        * Ensure that year and film title columns are clean and consistent (no footnotes, parentheses, etc.).
        * Save the results as best_picture.csv.  

In [2]:
# read in website for webscraping
URL = 'https://en.wikipedia.org/wiki/Academy_Award_for_Best_Picture'

headers = {
    "User-Agent": "MyPythonScraper"
}

response = requests.get(URL, headers=headers)

In [3]:
# check for connection
response.status_code

200

In [4]:
# create object for webscrape
movies_soup = BeautifulSoup(response.text)

In [5]:
# locate tables in HTML
movies_soup.findAll('table', attrs={'class' : 'wikitable'})[0]

<table class="wikitable sortable sticky-header" style="font-size:1.00em; line-height:1.5em;">
<tbody><tr bgcolor="#bebebe">
<th width="5%">Year of Film Release
</th>
<th width="40%">Film
</th>
<th width="55%">Film Studio
</th></tr>
<tr style="background:#FAEB86">
<th rowspan="3"><a href="/wiki/1928_in_film" title="1928 in film">1927/28</a><br/><span style="font-size: 85%;"><a href="/wiki/1st_Academy_Awards" title="1st Academy Awards">(1st)</a></span>
</th>
<td><i><b><a href="/wiki/Wings_(1927_film)" title="Wings (1927 film)">Wings</a></b></i>
</td>
<td><b><a href="/wiki/Famous_Players%E2%80%93Lasky" title="Famous Players–Lasky">Famous Players–Lasky</a> <span style="font-size: 85%;">(<a href="/wiki/Lucien_Hubbard" title="Lucien Hubbard">Lucien Hubbard</a>, <a href="/wiki/Jesse_L._Lasky" title="Jesse L. Lasky">Jesse L. Lasky</a>, <a href="/wiki/B._P._Schulberg" title="B. P. Schulberg">B. P. Schulberg</a>, &amp; <a href="/wiki/Adolph_Zukor" title="Adolph Zukor">Adolph Zukor</a>, producers

In [6]:
# display tables that we are scraping
table_html = str(movies_soup.findAll('table', attrs={'class' : 'wikitable'}))
from IPython.core.display import HTML
HTML(table_html)

In [7]:
# display tables as a list of dataframes 
pd.read_html(StringIO(str(movies_soup.findAll('table', attrs={'class' : 'wikitable'}))))[0]

,Year of Film Release,Film,Film Studio
0,1927/28 (1st),Wings,"Famous Players–Lasky (Lucien Hubbard, Jesse L...."
1,1927/28 (1st),7th Heaven,"Fox (William Fox, producer)"
2,1927/28 (1st),The Racket,"The Caddo Company (Howard Hughes, producer)"
3,1928/29 (2nd) [a],NaN,NaN
4,1928/29 (2nd) [a],The Broadway Melody,Metro-Goldwyn-Mayer (Irving Thalberg & Lawrenc...
5,1928/29 (2nd) [a],Alibi,"Feature Productions (Roland West, producer)"
6,1928/29 (2nd) [a],The Hollywood Revue,Metro-Goldwyn-Mayer (Irving Thalberg & Harry R...
7,1928/29 (2nd) [a],In Old Arizona,"Fox (Winfield Sheehan, producer)"
8,1928/29 (2nd) [a],The Patriot,Paramount Famous Lasky


In [8]:
# concatenate all tables into one dataframe
tables = movies_soup.findAll('table', attrs={'class' : 'wikitable'})

all_movies_df = pd.concat(
    [pd.read_html(StringIO(str(table)))[0] 
     for table in tables], ignore_index=True)
all_movies_df.head()


,Year of Film Release,Film,Film Studio,Film Studio/Producer(s),Producer(s),Record,Producer,Age,Production company/distributor,Nominations,Wins
0,1927/28 (1st),Wings,"Famous Players–Lasky (Lucien Hubbard, Jesse L....",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1927/28 (1st),7th Heaven,"Fox (William Fox, producer)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1927/28 (1st),The Racket,"The Caddo Company (Howard Hughes, producer)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1928/29 (2nd) [a],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1928/29 (2nd) [a],The Broadway Melody,Metro-Goldwyn-Mayer (Irving Thalberg & Lawrenc...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# locate winning movie titles for new column 
movie_wins_html = movies_soup.findAll('tr', attrs={'style' : 'background:#FAEB86'})
winning_movies = re.findall(r'title="(.+?)"', str(movie_wins_html))
winning_movies_final = [re.sub(r'\(.*?\)', '', m).strip() for m in winning_movies]
winning_movies_final[:10]

['1928 in film',
 '1st Academy Awards',
 'Wings',
 'Famous Players–Lasky',
 'Lucien Hubbard',
 'Jesse L. Lasky',
 'B. P. Schulberg',
 'Adolph Zukor',
 'The Broadway Melody',
 'Metro-Goldwyn-Mayer']

In [10]:
# add winner column to dataframe and use 'winning_movies_final' to filter for "Yes/No"
all_movies_df['Winner'] = all_movies_df['Film'].isin(winning_movies_final)
all_movies_df['Winner'] = all_movies_df['Winner'].map({True : 'Yes', False : 'No'})
all_movies_df.head()

,Year of Film Release,Film,Film Studio,Film Studio/Producer(s),Producer(s),Record,Producer,Age,Production company/distributor,Nominations,Wins,Winner
0,1927/28 (1st),Wings,"Famous Players–Lasky (Lucien Hubbard, Jesse L....",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes
1,1927/28 (1st),7th Heaven,"Fox (William Fox, producer)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
2,1927/28 (1st),The Racket,"The Caddo Company (Howard Hughes, producer)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
3,1928/29 (2nd) [a],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
4,1928/29 (2nd) [a],The Broadway Melody,Metro-Goldwyn-Mayer (Irving Thalberg & Lawrenc...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes


In [11]:
# display column names for column drop
all_movies_df.columns

Index(['Year of Film Release', 'Film', 'Film Studio',
       'Film Studio/Producer(s)', 'Producer(s)', 'Record', 'Producer', 'Age',
       'Production company/distributor', 'Nominations', 'Wins', 'Winner'],
      dtype='object')

In [12]:
# drop unnecesary columns and NaN values
all_movies_dropped_columns = all_movies_df.drop(['Film Studio', 'Film Studio/Producer(s)', 'Producer(s)', 'Record', 'Producer', 'Age', 'Production company/distributor', 'Nominations', 'Wins'], axis=1) 
all_movies_dropped_columns_rename = all_movies_dropped_columns.rename(columns={'Year of Film Release': 'Year'})
dropna_df = all_movies_dropped_columns_rename.dropna().copy()
dropna_df.tail(2)

,Year,Film,Winner
689,2024 (97th),The Substance,No
690,2024 (97th),Wicked,No


In [16]:
# clean year values in year column
dropna_df['Year'] = dropna_df['Year'].str.extract(r"(\d{4})")
movies = dropna_df

In [14]:
movies

,Year,Film,Winner
0,1927,Wings,Yes
1,1927,7th Heaven,No
2,1927,The Racket,No
4,1928,The Broadway Melody,Yes
5,1928,Alibi,No
...,...,...,...
686,2024,Emilia Pérez,No
687,2024,I'm Still Here,No
688,2024,Nickel Boys,No
689,2024,The Substance,No


2. Gather Movie Data via TMDB API  
    a. Set up the API    
    * Create a free [TMDB account](https://developer.themoviedb.org/docs/getting-started)  
    * Generate an API key are review their documentation, especially:  
        * /discover/movie  
        * /movie/{movie_id}  
        * /search/movie  
    b. Collect top movies (2015-2024)  
    For each year from 2015 to 2024:  
        * Query TMDB for the top 100 movies (by vote count).  
        * For each movie, gather:  
            * Title  
            * Release Year  
            * Genre(s)  
            * Vote Average  
            * Vote Count  
            * Budget  
            * Revenue  
            * TMDB ID  
        * Store all results in a single DataFrame and export to movies_2015_2024.csv.
        * Hint: TMDB rate limits are generous for free accounts, but you should pause between requests (eg. time.sleep(0.25)). 
        * Some Oscar films may not appear in the top 100 by vote count. For any missing, use the /search/movie endpoint to add it.  

**Optional Extension: Actors and Actresses** 

1. Scrape Wikipedia for Best Actor and Best Actress Data
    * Scrape the following Wikipedia pages:  
        * [Best Actor](https://en.wikipedia.org/wiki/Academy_Award_for_Best_Actor)
        * [Best Actress](https://en.wikipedia.org/wiki/Academy_Award_for_Best_Actress)
    * Each apge contains tables of winners and nominees by year.
    * Extract the following columns:  
        * Year
        * Actor/Actress Name
        * Film Title
        * Winner (Yes/No)
    * Data cleaning tips:  
        * Remove footnote markers from names and movie titles.
        * Ensure that you save just the release year (eg. 2009 instead of 2009 (82nd))
        * Store the cleaned data as two csv files:  
            * best_actor.csv
            * best_actress.csv  

2. Collect Actor and Actress Filmographies  
    Using the data from your actor and actresses CSVs:  
    * Search TMDB for each recent performer (using /search/person). Note: you can start with 2015-2024 initially, but, if time allows, you can go back even further.
    * For each person, retrieve their movie credits using /person/{person_id}/movie_credits.  
    * Extract relevant fields for each movie, such as:  
        * Actor/Actress Name  
        * Movie Title  
        * Character Name (optional)  
        * Release Year  
        * Movie ID
    * Combine all filmographies into one file, actor_filmography.csv

In [15]:
# http://localhost